# Import Package

In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from PIL import Image, ImageFile,UnidentifiedImageError, ImageEnhance,ImageOps
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from scipy.stats import skew
import random
import torch
import seaborn as sns
import pickle
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# STEP 1:  Problem Understanding 

- Bài toán: Nhận diện ký hiệu cơ thể chuyển sang ngôn ngữ đọc nói từ ảnh và video có sẵn.

- Loại bài toán: Phân loại ký hiệu tay, xử lý chuỗi văn bản, chuyển văn bản sang giọng nói .

- Đầu vào:  Các ảnh và video bàn tay và cơ thể người đang thưc hiện các ký hiệu.

- Output : (target) đưa ra văn bản và giọng nói sau khi đã nhận diện được chuỗi hành dộng.

- Mục tiêu: Giúp người khiếm thính và người khiếm thị có thể giao tiếp tốt với mọi người xung quanh.

# STEP 2: Data Understanding.

Mục đích: Hiểu rõ chất lượng đặc điểm và tiền xử lý dữ liệu ảnh

Bao gồm 
- Inconsistent: Tìm dữ liệu không nhất quán
- Data Overview: Kiểm tra tổng quan dữ liệu( số lượng ảnh và kích thước phổ biến)
- Imbalanced: Kiểm tra sự mất cân bằng giữa các lớp ( A, B, C, ...)
- Data Augmentation : Tăng cường dữ liệu 

In [ ]:
# 2.1 Tìm dữ liệu không nhất quán 
dataset_path="../dataset"

# duyệt từng tệp trong từng thư mục con của dataset

for person in os.listdir(dataset_path): 
    person_path = os.path.join(dataset_path, person)
    for img_file in os.listdir(person_path):
        img_path = os.path.join(person_path, img_file)
        try:
            with Image.open(img_path) as img:
                img.verify()  # hàm xác minh ảnh
        except (UnidentifiedImageError, OSError):
            print(f"Ảnh lỗi: {img_path} \n Thực hiện xóa...")
            os.remove(img_path)  

In [ ]:
# kiểm tra số lượng ảnh và kích thước phổ biến
sizes = []
for person in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person)
    for img_file in os.listdir(person_path):
        img_path = os.path.join(person_path, img_file)
        try:
            with Image.open(img_path) as img:
                sizes.append(img.size)
        except:
            pass  # Bỏ qua ảnh lỗi đã phát hiện ở trên

print("Số lượng ảnh:", len(sizes))
print("Kích thước phổ biến nhất:", max(set(sizes), key=sizes.count))